In [3]:
from mpa.models import Mpa, mpa_post_save
from django.db.models.signals import post_save, post_delete
from mpa.views import mpas_all_nogeom
import reversion
from reversion.models import Revision
from reversion.models import Version
import datetime
from django.db.models import Q, F, Func

In [66]:
count = 0
for m in mpas_all_nogeom.filter(
        Q(country__icontains='IDN') | Q(sovereign__icontains='IDN')):
    versions = Version.objects.get_for_object(m)
    created = 'Never'
    if not m.wdpa_id: continue
    try:
        created = versions.first().revision.date_created
        if created > datetime.datetime(2014, 11, 5):
            count = count + 1
            print(count, created, 'count:', len(versions), 'mpa_id:', m.mpa_id, 'wdpa_id:', m.wdpa_id)
    except:
        pass

1 2017-08-25 10:49:19.388194 count: 4 mpa_id: 5430 wdpa_id: 555512132
2 2017-08-25 10:49:27.917067 count: 5 mpa_id: 5390 wdpa_id: 555511962
3 2014-12-08 22:20:37.629472 count: 2 mpa_id: 650 wdpa_id: 8704
4 2014-12-10 12:29:37.379074 count: 2 mpa_id: 1343 wdpa_id: 17867
5 2014-12-08 07:48:01.625311 count: 1 mpa_id: 67707531 wdpa_id: 555542902
6 2014-12-08 22:35:58.736499 count: 2 mpa_id: 3533 wdpa_id: 198429
7 2014-12-09 13:17:25.583623 count: 2 mpa_id: 688 wdpa_id: 8838
8 2014-11-05 16:33:58.216500 count: 1 mpa_id: 6033 wdpa_id: 10289
9 2014-11-05 16:35:01.940479 count: 1 mpa_id: 235 wdpa_id: 1900
10 2014-12-08 05:52:31.670049 count: 1 mpa_id: 67706002 wdpa_id: 8895
11 2014-11-05 16:33:56.406503 count: 1 mpa_id: 5989 wdpa_id: 8523
12 2014-12-08 18:50:48.169405 count: 2 mpa_id: 2622 wdpa_id: 101486
13 2014-12-09 23:26:45.259726 count: 5 mpa_id: 6062 wdpa_id: 11923
14 2014-12-08 07:55:22.175608 count: 3 mpa_id: 5408 wdpa_id: 555511981
15 2014-12-08 12:44:25.820237 count: 2 mpa_id: 6909 w

134 2017-08-25 10:48:40.632029 count: 3 mpa_id: 1972 wdpa_id: 62418
135 2014-12-10 11:27:16.815951 count: 2 mpa_id: 5991 wdpa_id: 8588
136 2014-12-09 13:08:34.250648 count: 2 mpa_id: 5409 wdpa_id: 555511982
137 2014-11-05 16:34:01.168314 count: 1 mpa_id: 6145 wdpa_id: 12873
138 2014-12-08 10:28:57.892772 count: 2 mpa_id: 67705394 wdpa_id: 555511967
139 2015-11-16 11:06:42.525580 count: 2 mpa_id: 67701759 wdpa_id: 30012
140 2014-12-10 04:51:11.873823 count: 4 mpa_id: 6060 wdpa_id: 11901
141 2014-12-08 16:58:43.503181 count: 2 mpa_id: 5815 wdpa_id: 555542492
142 2014-12-09 23:16:26.770044 count: 2 mpa_id: 5421 wdpa_id: 555511995
143 2014-12-09 09:55:35.945744 count: 2 mpa_id: 5412 wdpa_id: 555511985
144 2014-12-09 23:04:57.656306 count: 2 mpa_id: 5411 wdpa_id: 555511984
145 2014-12-08 22:54:25.109781 count: 2 mpa_id: 5400 wdpa_id: 555511973
146 2014-12-09 12:23:00.766882 count: 5 mpa_id: 5391 wdpa_id: 555511964
147 2014-12-09 10:56:44.895577 count: 2 mpa_id: 5395 wdpa_id: 555511968
148 2

264 2015-06-16 10:19:52.541842 count: 2 mpa_id: 262 wdpa_id: 2102
265 2014-12-09 13:24:18.299976 count: 2 mpa_id: 656 wdpa_id: 8720
266 2014-11-05 16:33:56.614847 count: 1 mpa_id: 5994 wdpa_id: 8648
267 2014-12-10 11:18:05.546460 count: 2 mpa_id: 5878 wdpa_id: 3031
268 2015-06-16 10:19:41.212589 count: 2 mpa_id: 2144 wdpa_id: 67725
269 2014-11-05 16:34:26.061187 count: 1 mpa_id: 7531 wdpa_id: 555542902
270 2014-11-05 16:35:56.746961 count: 1 mpa_id: 613 wdpa_id: 8072
271 2015-05-20 14:10:39.066823 count: 1 mpa_id: 67702260 wdpa_id: 68011
272 2014-12-08 22:59:53.995535 count: 2 mpa_id: 695 wdpa_id: 8871
273 2014-12-10 11:12:35.506262 count: 2 mpa_id: 617 wdpa_id: 8107
274 2014-12-09 13:26:32.164716 count: 2 mpa_id: 660 wdpa_id: 8735
275 2014-12-10 11:12:16.664493 count: 2 mpa_id: 616 wdpa_id: 8106
276 2014-12-11 09:28:39.557971 count: 3 mpa_id: 5386 wdpa_id: 555511956
277 2014-12-10 09:51:14.572133 count: 2 mpa_id: 626 wdpa_id: 8525
278 2014-12-10 09:51:38.886830 count: 2 mpa_id: 654 wd

In [2]:
from importlib import reload
from wdpa import merge
reload(merge)

<module 'wdpa.merge' from '/Users/russmo/Code/mpatlas-dj2/mpatlas/wdpa/merge.py'>

In [96]:
wdpa2remove = merge.getRemoveWdpaList()

In [97]:
len(wdpa2remove)

0

In [72]:
rm_mpas = Mpa.objects.filter(wdpa_id__in=wdpa2remove).order_by('country')
len(rm_mpas)

125

In [73]:
for m in rm_mpas:
    print(m.mpa_id, m.wdpa_id, m.country, m.sovereign, m.is_mpa, m.no_take, m.no_take_area, m.name, m.designation_eng)

5390 555511962 IDN IDN True Part 1220.0 Raja Ampat (Selat Dampier) District Marine Conservation Area
6062 11923 IDN IDN True Not Reported 0.0 Kepulauan Sepanjang (Kab Sumenep) District Marine Conservation Area
9790 62484 IDN IDN True Not Reported 0.0 Saobi (Kangean) Nature Reserve
5396 555511969 IDN IDN True Not Reported 0.0 Sumba Strait Marine Area (Laut Sawu) National Park
4826 315475 IDN IDN True Not Reported 0.0 Pulau Anggrameos Wildlife Sanctuary
6004 9003 IDN IDN True Not Reported 0.0 Pulau Weh Marine Nature Recreational Park
706 8903 IDN IDN True Not Reported 0.0 Kuala Jambu Aye Nature Reserve
649 8696 IDN IDN True Not Reported 0.0 Muara Kayan Nature Reserve
705 8898 IDN IDN True Not Reported 0.0 Kuala Langsa Nature Reserve
701 8881 IDN IDN False Not Reported 0.0 Tanjung Batikolo Game Reserve
714 8923 IDN IDN False Not Reported 0.0 Kuala Tapas Nature Reserve
723 8964 IDN IDN False Not Reported 0.0 Bakau Selat Dumai Game Reserve
668 8767 IDN IDN False Not Reported 0.0 Gunung Kela

In [95]:
merge.removeMpasByWdpaId(wdpa2remove)

Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Access denied
Carto Error for getting mpa_ids Ac

In [6]:
reload(merge)

<module 'wdpa.merge' from '/Users/russmo/Code/mpatlas-dj2/mpatlas/wdpa/merge.py'>

In [98]:
wdpa2add = merge.getAddWdpaList()

In [99]:
len(wdpa2add)

90

In [103]:
wdpapid2add = merge.getAddWdpaPidList(verbose=True)

In [102]:
len(wdpapid2add)

0

In [10]:
zone1 = Wdpa2018Poly.objects.filter(wdpa_pid='555599633').get()
print(zone1.wdpaid, zone1.wdpa_pid)
zw = zone1.wdpaid

166954.0 555599633


In [14]:
wp = Wdpa2018Poly.objects.filter(wdpaid=zw)

In [15]:
wzones = []
mpasw = [float(wi) for wi in mpas_all_nogeom.filter(wdpa_id__isnull=False).order_by('wdpa_id').values_list('wdpa_id', flat=True)]
polyset = Wdpa2018Poly.objects.filter(wdpaid__in=mpasw).only('wdpaid', 'wdpa_pid', 'name', 'desig_eng', 'iso3', 'parent_iso3')
for w in polyset:
    if w.wdpa_pid != str(int(w.wdpaid)):
        wzones.append(w.wdpaid)
wzones = list(set(wzones))
len(wzones)

362

In [181]:
count = 0
for wid in wzones:
    zones = Wdpa2018Poly.objects.filter(wdpaid=wid).only('wdpaid', 'wdpa_pid', 'name', 'desig_eng', 'iso3', 'parent_iso3')
    firstpid = zones.order_by('wdpa_pid').first().wdpa_pid
    mpaz = mpas_all_nogeom.filter(wdpa_id=int(wid))
    for m in mpaz:
        print(m.mpa_id, m.wdpa_id, m.wdpa_pid)
        m.wdpa_pid = firstpid
        count += 1
        print(count, m.mpa_id, m.wdpa_id, m.wdpa_pid)
        m.save()


1389 18432 None
1 1389 18432 18432_A
Carto Error for mpa_id 1389: Access denied
14978 555556866 None
2 14978 555556866 555556866_A
Carto Error for mpa_id 14978: Access denied
14980 555556868 None
3 14980 555556868 555556868_A
Carto Error for mpa_id 14980: Access denied
3874 305157 None
4 3874 305157 305157_A
Carto Error for mpa_id 3874: Access denied
14982 555556870 None
5 14982 555556870 555556870_A
Carto Error for mpa_id 14982: Access denied
14983 555556871 None
6 14983 555556871 555556871_A
Carto Error for mpa_id 14983: Access denied
14986 555556874 None
7 14986 555556874 555556874_A
Carto Error for mpa_id 14986: Access denied
9898 126987 None
8 9898 126987 126987_A
Carto Error for mpa_id 9898: Access denied
14987 555556875 None
9 14987 555556875 555556875_A
Carto Error for mpa_id 14987: Access denied
14988 555556876 None
10 14988 555556876 555556876_A
Carto Error for mpa_id 14988: Access denied
14989 555556877 None
11 14989 555556877 555556877_A
Carto Error for mpa_id 14989: Access

GEOS_NOTICE: Ring Self-intersection at or near point -153.45194612099996 -76.920097581999983

GEOS_NOTICE: Ring Self-intersection at or near point -153.45194612099996 -76.920097581999983



Carto Error for mpa_id 9047: Access denied
68808296 555624810 None
72 68808296 555624810 555624810_A
Carto Error for mpa_id 68808296: Access denied
68808295 555624810 None
73 68808295 555624810 555624810_A
Carto Error for mpa_id 68808295: Access denied
5691 555516292 None
74 5691 555516292 555516292_A
Carto Error for mpa_id 5691: Access denied
5692 555516295 None
75 5692 555516295 555516295_A
Carto Error for mpa_id 5692: Access denied
5693 555516314 None
76 5693 555516314 555516314_A
Carto Error for mpa_id 5693: Access denied
5694 555516319 None
77 5694 555516319 555516319_A
Carto Error for mpa_id 5694: Access denied
5696 555516352 None
78 5696 555516352 555516352_A
Carto Error for mpa_id 5696: Access denied
5697 555516353 None
79 5697 555516353 555516353_A
Carto Error for mpa_id 5697: Access denied
5698 555516354 None
80 5698 555516354 555516354_A
Carto Error for mpa_id 5698: Access denied
5699 555516355 None
81 5699 555516355 555516355_A
Carto Error for mpa_id 5699: Access denied
570

GEOS_NOTICE: Ring Self-intersection at or near point 152.19999999900085 -22.0000179990314

GEOS_NOTICE: Ring Self-intersection at or near point 152.19999999900085 -22.0000179990314



Carto Error for mpa_id 301: Access denied
7700301 2628 None
142 7700301 2628 2628_A


...

Carto Error for mpa_id 68808378: Access denied
68808380 2628 None
145 68808380 2628 2628_A


GEOS_NOTICE: Ring Self-intersection at or near point 149.01113309532235 -20.234186196388421

GEOS_NOTICE: Ring Self-intersection at or near point 149.01113309532235 -20.234186196388421



Carto Error for mpa_id 68808380: Access denied
68808375 2628 None
146 68808375 2628 2628_A
Carto Error for mpa_id 68808375: Access denied
68808379 2628 None
147 68808379 2628 2628_A
Carto Error for mpa_id 68808379: Access denied
68808377 2628 None
148 68808377 2628 2628_A
Carto Error for mpa_id 68808377: Access denied
86 584 None
149 86 584 584_A
Carto Error for mpa_id 86: Access denied
1889 39501 None
150 1889 39501 39501_A
Carto Error for mpa_id 1889: Access denied
2047 64093 None
151 2047 64093 64093_A
Carto Error for mpa_id 2047: Access denied
7700087 613 None
152 7700087 613 613_A
Carto Error for mpa_id 7700087: Access denied
89 628 None
153 89 628 628_A
Carto Error for mpa_id 89: Access denied
91 632 None
154 91 632 632_A
Carto Error for mpa_id 91: Access denied
2052 64122 None
155 2052 64122 64122_A
Carto Error for mpa_id 2052: Access denied
1080 13100 None
156 1080 13100 13100_A
Carto Error for mpa_id 1080: Access denied
1081 13101 None
157 1081 13101 13101_A
Carto Error for mp

Carto Error for mpa_id 7704337: Access denied
2092 67031 None
242 2092 67031 67031_A
Carto Error for mpa_id 2092: Access denied
2093 67038 None
243 2093 67038 67038_A
Carto Error for mpa_id 2093: Access denied
2599 99806 None
244 2599 99806 99806_A
Carto Error for mpa_id 2599: Access denied
2094 67041 None
245 2094 67041 67041_A
Carto Error for mpa_id 2094: Access denied
2095 67042 None
246 2095 67042 67042_A
Carto Error for mpa_id 2095: Access denied
2096 67043 None
247 2096 67043 67043_A
Carto Error for mpa_id 2096: Access denied
2097 67050 None
248 2097 67050 67050_A
Carto Error for mpa_id 2097: Access denied
2098 67051 None
249 2098 67051 67051_A
Carto Error for mpa_id 2098: Access denied
2099 67052 None
250 2099 67052 67052_A
Carto Error for mpa_id 2099: Access denied
2100 67054 None
251 2100 67054 67054_A
Carto Error for mpa_id 2100: Access denied
2101 67055 None
252 2101 67055 67055_A
Carto Error for mpa_id 2101: Access denied
2102 67056 None
253 2102 67056 67056_A
Carto Error f

Carto Error for mpa_id 5674: Access denied
1362 18161 None
335 1362 18161 18161_A
Carto Error for mpa_id 1362: Access denied
1363 18164 None
336 1363 18164 18164_A
Carto Error for mpa_id 1363: Access denied
1364 18166 None
337 1364 18166 18166_A
Carto Error for mpa_id 1364: Access denied
2070 65277 None
338 2070 65277 65277_A
Carto Error for mpa_id 2070: Access denied
1365 18178 None
339 1365 18178 18178_A
Carto Error for mpa_id 1365: Access denied
2071 65294 None
340 2071 65294 65294_A
Carto Error for mpa_id 2071: Access denied
12458 555515666 None
341 12458 555515666 555515666_A
Carto Error for mpa_id 12458: Access denied
9812 67347 None
342 9812 67347 67347_A
Carto Error for mpa_id 9812: Access denied
9751 18197 None
343 9751 18197 18197_A
Carto Error for mpa_id 9751: Access denied
2135 67351 None
344 2135 67351 67351_A
Carto Error for mpa_id 2135: Access denied
2136 67352 None
345 2136 67352 67352_A
Carto Error for mpa_id 2136: Access denied
2137 67356 None
346 2137 67356 67356_A
C

In [184]:
from django.db.models import F, Func
mpazz = mpas_all_nogeom.filter(wdpa_id__isnull=False).exlude(wdpa_pid__in=wd)
print(mpazz.count())
mpazz.update(wdpa_pid=Func(F('wdpa_id'), function='CAST', template='%(function)s(%(expressions)s as varchar(52))'))

13030


13030

In [203]:
mpazx = mpas_all_nogeom.filter(wdpa_id__isnull=False).exclude(wdpa_id__in=[int(w) for w in wzones])
mpazx.update(wdpa_pid=Func(F('wdpa_id'), function='CAST', template='%(function)s(%(expressions)s as varchar(52))'))

13030

In [16]:
count = 0
mpazy = mpas_all_nogeom.only('wdpa_id', 'mpa_id', 'wdpa_pid').filter(wdpa_id__isnull=False).exclude(wdpa_id__in=[int(w) for w in wzones])
for m in mpazy:
    count += 1
    mywid = m.wdpa_id
    mypid = str(m.wdpa_id)
    myid = m.mpa_id
    m.wdpa_pid = mypid
    print(count, m.mpa_id, m.wdpa_id, m.wdpa_pid)
    #m.save()
    mq = mpas_all_nogeom.filter(mpa_id=myid).update(wdpa_pid=mypid)

1 9550 555547644 555547644
2 7431 902834 902834
3 9522 555547616 555547616
4 3312 166937 166937
5 9551 555547645 555547645
6 6680 62917 62917
7 6806 101752 101752
8 3946 305726 305726
9 3929 305642 305642
10 6177 13214 13214
11 9509 555547603 555547603
12 9787 62101 62101
13 6745 71026 71026
14 1370 18273 18273
15 9510 555547604 555547604
16 7161 316916 316916
17 5762 555522511 555522511
18 6014 9747 9747
19 15305 555571471 555571471
20 9560 555547654 555547654
21 67701792 31288 31288
22 9789 62438 62438
23 14538 555547891 555547891
24 9511 555547605 555547605
25 14822 555549291 555549291
26 934 12267 12267
27 9512 555547606 555547606
28 6258 13940 13940
29 7018 305781 305781
30 15319 555571485 555571485
31 67706274 13974 13974
32 3593 220205 220205
33 6021 9810 9810
34 3610 220223 220223
35 9593 555547687 555547687
36 14408 555547759 555547759
37 7705496 555512237 555512237
38 3574 220013 220013
39 7021 305839 305839
40 14536 555547889 555547889
41 14523 555547876 555547876
42 10470 1

372 7701198 14553 14553
373 3013 151820 151820
374 7701046 12961 12961
375 2368 68264 68264
376 2191 67877 67877
377 6872 166875 166875
378 10908 182810 182810
379 12877 555527680 555527680
380 15061 555559198 555559198
381 7317 900811 900811
382 15101 555559309 555559309
383 3878 305233 305233
384 3087 152292 152292
385 2831 127872 127872
386 11247 309286 309286
387 6992 305221 305221
388 12082 392995 392995
389 14651 555548864 555548864
390 5341 902304 902304
391 15986 555579772 555579772
392 15973 555579656 555579656
393 15959 555579625 555579625
394 12680 555523850 555523850
395 5255 478050 478050
396 2206 67892 67892
397 13464 555534844 555534844
398 7141 315562 315562
399 3713 302836 302836
400 13463 555534843 555534843
401 5349 902368 902368
402 5366 903139 903139
403 3503 198303 198303
404 5428 555512062 555512062
405 10435 156230 156230
406 14551 555547976 555547976
407 7525 555542772 555542772
408 2319 68135 68135
409 6897 198346 198346
410 2673 102877 102877
411 3635 300327 

712 11501 341329 341329
713 6874 166885 166885
714 15825 555578730 555578730
715 13208 555529968 555529968
716 15814 555578717 555578717
717 15815 555578718 555578718
718 15304 555571470 555571470
719 15812 555578715 555578715
720 6037 10720 10720
721 13625 555538230 555538230
722 15762 555578665 555578665
723 15804 555578707 555578707
724 15803 555578706 555578706
725 3927 305640 305640
726 15789 555578692 555578692
727 15342 555571706 555571706
728 5977 5191 5191
729 15795 555578698 555578698
730 15119 555559498 555559498
731 15808 555578711 555578711
732 15787 555578690 555578690
733 3256 156334 156334
734 15786 555578689 555578689
735 5586 555512401 555512401
736 15785 555578688 555578688
737 16052 555580663 555580663
738 15784 555578687 555578687
739 15783 555578686 555578686
740 15118 555559486 555559486
741 15775 555578678 555578678
742 15632 555577707 555577707
743 15777 555578680 555578680
744 15772 555578675 555578675
745 15771 555578674 555578674
746 15774 555578677 55557867

1100 6477 26472 26472
1101 10902 181595 181595
1102 11612 349996 349996
1103 9827 81191 81191
1104 3058 152118 152118
1105 10904 182720 182720
1106 5258 478053 478053
1107 3029 151909 151909
1108 15756 555578659 555578659
1109 10903 182719 182719
1110 9903 146760 146760
1111 15894 555578838 555578838
1112 15903 555578867 555578867
1113 3923 305594 305594
1114 671 8770 8770
1115 9906 147435 147435
1116 10873 178988 178988
1117 9901 146753 146753
1118 3979 305873 305873
1119 4728 313251 313251
1120 11212 308375 308375
1121 4341 309277 309277
1122 11214 308379 308379
1123 3403 182734 182734
1124 3962 305813 305813
1125 9793 63669 63669
1126 9825 81115 81115
1127 6964 220108 220108
1128 15798 555578701 555578701
1129 5777 555529056 555529056
1130 15818 555578722 555578722
1131 2750 126143 126143
1132 15239 555567617 555567617
1133 11500 341325 341325
1134 7247 365005 365005
1135 6729 68241 68241
1136 1748 29782 29782
1137 5630 555512445 555512445
1138 3106 152442 152442
1139 4897 349993 34

1489 13517 555535203 555535203
1490 12144 395240 395240
1491 13513 555535197 555535197
1492 13839 555540257 555540257
1493 12748 555525902 555525902
1494 13053 555528724 555528724
1495 12142 395189 395189
1496 6811 101775 101775
1497 12132 394901 394901
1498 11171 196471 196471
1499 12135 394908 394908
1500 10871 178865 178865
1501 14182 555543240 555543240
1502 15629 555577677 555577677
1503 7701017 12906 12906
1504 14104 555543160 555543160
1505 14101 555543157 555543157
1506 14062 555543118 555543118
1507 13571 555538015 555538015
1508 14066 555543122 555543122
1509 14063 555543119 555543119
1510 14060 555543116 555543116
1511 14061 555543117 555543117
1512 262 2102 2102
1513 13563 555538002 555538002
1514 13555 555537990 555537990
1515 13556 555537991 555537991
1516 11891 390556 390556
1517 7700300 2627 2627
1518 12543 555522359 555522359
1519 12597 555522514 555522514
1520 12552 555522375 555522375
1521 12551 555522374 555522374
1522 12504 555517947 555517947
1523 12545 555522361 

...

11309 14842 555550524 555550524
11310 14748 555549217 555549217
11311 6142 12840 12840
11312 6794 97464 97464
11313 6795 97466 97466
11314 907 11848 11848
11315 11855 388701 388701
11316 14839 555550517 555550517
11317 14164 555543220 555543220
11318 6454 19651 19651
11319 6456 19657 19657
11320 14172 555543228 555543228
11321 6118 12041 12041
11322 6115 12038 12038
11323 6116 12039 12039
11324 14163 555543219 555543219
11325 6285 14079 14079
11326 14160 555543216 555543216
11327 6125 12705 12705
11328 7243 342653 342653
11329 11623 354835 354835
11330 14162 555543218 555543218
11331 6288 14183 14183
11332 6289 14187 14187
11333 12611 555522551 555522551
11334 12612 555522558 555522558
11335 12631 555522829 555522829
11336 9466 28491 28491
11337 12432 555515267 555515267
11338 12313 398096 398096
11339 12314 398101 398101
11340 14727 555549196 555549196
11341 12430 555515254 555515254
11342 15060 555559197 555559197
11343 6792 97455 97455
11344 6793 97463 97463
11345 12249 397445 39744

11676 2726 108073 108073
11677 4837 315488 315488
11678 9881 125867 125867
11679 9879 125861 125861
11680 6476 26470 26470
11681 9874 125848 125848
11682 12988 555527992 555527992
11683 9875 125850 125850
11684 3431 183090 183090
11685 13700 555539376 555539376
11686 12987 555527991 555527991
11687 14762 555549231 555549231
11688 3487 193469 193469
11689 9734 7534 7534
11690 11395 330745 330745
11691 15193 555567534 555567534
11692 9694 831 831
11693 3808 303698 303698
11694 3481 193456 193456
11695 3485 193462 193462
11696 3468 193440 193440
11697 3484 193461 193461
11698 3459 183123 183123
11699 3477 193450 193450
11700 14704 555549173 555549173
11701 3474 193447 193447
11702 10355 156139 156139
11703 12703 555524064 555524064
11704 3443 183106 183106
11705 3453 183117 183117
11706 14767 555549236 555549236
11707 3463 183154 183154
11708 7036 305927 305927
11709 5650 555512466 555512466
11710 3424 183070 183070
11711 3416 183061 183061
11712 3417 183062 183062
11713 1245 15775 15775


12084 3008 151782 151782
12085 10123 153425 153425
12086 10104 153116 153116
12087 9878 125859 125859
12088 9877 125858 125858
12089 10058 150970 150970
12090 6429 18304 18304
12091 10082 151137 151137
12092 10066 151016 151016
12093 6882 166902 166902
12094 2907 145579 145579
12095 2980 151520 151520
12096 10076 151076 151076
12097 9982 150218 150218
12098 10040 150790 150790
12099 5760 555522506 555522506
12100 3977 305869 305869
12101 10001 150414 150414
12102 10013 150541 150541
12103 10009 150488 150488
12104 3974 305850 305850
12105 9981 150182 150182
12106 9952 149830 149830
12107 3169 152806 152806
12108 3382 173501 173501
12109 12433 555515272 555515272
12110 3352 173030 173030
12111 12166 396460 396460
12112 9896 125913 125913
12113 9886 125892 125892
12114 3281 156590 156590
12115 3368 173319 173319
12116 3374 173361 173361
12117 5767 555522530 555522530
12118 3372 173325 173325
12119 3276 156571 156571
12120 5156 354196 354196
12121 3364 173302 173302
12122 11694 379209 379

12439 2853 136393 136393
12440 3345 169804 169804
12441 2862 137737 137737
12442 899 11723 11723
12443 7702366 68262 68262
12444 7702365 68253 68253
12445 191 1458 1458
12446 7706871 166871 166871
12447 7702522 95293 95293
12448 7702519 95290 95290
12449 7706775 95304 95304
12450 3342 169633 169633
12451 12640 555523038 555523038
12452 7706937 220053 220053
12453 7706934 220045 220045
12454 7706935 220048 220048
12455 6537 31420 31420
12456 7706949 220077 220077
12457 7706936 220049 220049
12458 7702532 95308 95308
12459 7702524 95297 95297
12460 7706950 220079 220079
12461 9566 555547660 555547660
12462 2857 136957 136957
12463 7706843 134954 134954
12464 192 1461 1461
12465 7703586 220086 220086
12466 7702363 68247 68247
12467 7704816 315046 315046
12468 8802017 63243 63243
12469 7706847 145498 145498
12470 7703582 220052 220052
12471 7702837 127889 127889
12472 6535 31417 31417
12473 6167 13203 13203
12474 2861 137703 137703
12475 7706134 12804 12804
12476 7702362 68243 68243
12477 

12814 7135 315547 315547
12815 8801666 26466 26466
12816 2371 68276 68276
12817 7700930 12243 12243
12818 68808216 555594026 555594026
12819 7422 902800 902800
12820 4248 307920 307920
12821 12321 400010 400010
12822 6438 18930 18930
12823 14975 555556768 555556768
12824 6843 134954 134954
12825 5909 3766 3766
12826 67700687 8835 8835
12827 68808213 555594010 555594010
12828 2532 95308 95308
12829 742 9281 9281
12830 7314 900793 900793
12831 6934 220045 220045
12832 15610 555577314 555577314
12833 2524 95297 95297
12834 6948 220076 220076
12835 2300 68103 68103
12836 2517 94085 94085
12837 15607 555577305 555577305
12838 15601 555577292 555577292
12839 6807 101767 101767
12840 15605 555577299 555577299
12841 7704019 305993 305993
12842 5771 555526877 555526877
12843 15596 555577263 555577263
12844 6435 18927 18927
12845 3564 204584 204584
12846 1753 29808 29808
12847 2388 68324 68324
12848 2367 68263 68263
12849 2530 95303 95303
12850 15604 555577296 555577296
12851 67700244 1939 1939


In [104]:
allpid2add = wdpapid2add + [str(int(i)) for i in wdpa2add]
len(allpid2add)

90

In [114]:
reload(merge)
addpoint_log = '/Users/russmo/Code/wdpa/log/addpoint_IDN_log.json'
addpoly_log = '/Users/russmo/Code/wdpa/log/addpoly_IDN_log.json'

In [112]:
points = Wdpa2018Point.objects.filter(wdpa_pid__in=allpid2add).order_by('wdpa_pid')
merge.updateMpasFromWdpaQueryset(qs=points, poly=False, logfile=addpoint_log, dryrun=True)

1/4 adding/updating wdpa_pid 19714 with mpa_id 68808483
ERROR processing mpa 68808483:  'NoneType' object has no attribute 'hexewkb'
2/4 adding/updating wdpa_pid 62845 with mpa_id 68808515
ERROR processing mpa 68808515:  'NoneType' object has no attribute 'hexewkb'
3/4 adding/updating wdpa_pid 9005 with mpa_id 68808516
ERROR processing mpa 68808516:  'NoneType' object has no attribute 'hexewkb'
4/4 adding/updating wdpa_pid 9235 with mpa_id 68808517
ERROR processing mpa 68808517:  'NoneType' object has no attribute 'hexewkb'


True

In [117]:
# post_save.disconnect(mpa_post_save, sender=Mpa)
polys = Wdpa2018Poly.objects.filter(wdpa_pid__in=allpid2add).order_by('wdpa_pid')
merge.updateMpasFromWdpaQueryset(qs=polys, poly=True, logfile=addpoly_log, dryrun=True)
# post_save.connect(mpa_post_save, sender=Mpa)

1/86 adding/updating wdpa_pid 101824 with mpa_id 68808649
    PID: 101824 has 1 field diffs  and GEOM diff
2/86 adding/updating wdpa_pid 17875 with mpa_id 68808650
3/86 adding/updating wdpa_pid 198298 with mpa_id 68808651
4/86 adding/updating wdpa_pid 317202 with mpa_id 68808652
    PID: 317202 has 1 field diffs  and GEOM diff
5/86 adding/updating wdpa_pid 317206 with mpa_id 68808653
    PID: 317206 has 1 field diffs  and GEOM diff
6/86 adding/updating wdpa_pid 317270 with mpa_id 68808654
    PID: 317270 has 1 field diffs  and GEOM diff
7/86 adding/updating wdpa_pid 317275 with mpa_id 68808655
    PID: 317275 has 1 field diffs  and GEOM diff
8/86 adding/updating wdpa_pid 555571189 with mpa_id 68808656
9/86 adding/updating wdpa_pid 555571198 with mpa_id 68808657
    PID: 555571198 has 1 field diffs  and GEOM diff
10/86 adding/updating wdpa_pid 555571200 with mpa_id 68808658
    PID: 555571200 has 1 field diffs  and GEOM diff
11/86 adding/updating wdpa_pid 555571229 with mpa_id 68808659


True

In [122]:
reload(merge)
updatepoint_log = '/Users/russmo/Code/wdpa/log/updatepoint_IDN_log.json'
updatepoly_log = '/Users/russmo/Code/wdpa/log/updatepoly_IDN_log.json'

In [125]:
wdpa_filter = (
    Q(iso3__icontains='IDN') | Q(parent_iso3__icontains='IDN')
)
polys = Wdpa2018Poly.objects.exclude(wdpa_pid__in=allpid2add).filter(wdpa_filter).filter(marine__in=('1','2')).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)
points = Wdpa2018Point.objects.exclude(wdpa_pid__in=allpid2add).filter(wdpa_filter).filter(marine__in=('1','2')).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)
allpid2update = list(set(list(polys) + list(points)))



In [126]:
len(allpid2update)

162

In [128]:
updatepoints = Wdpa2018Point.objects.filter(wdpa_pid__in=allpid2update)
merge.updateMpasFromWdpaQueryset(qs=updatepoints, poly=False, logfile=updatepoint_log, dryrun=True)

1/17 adding/updating wdpa_pid 555542726 with mpa_id 7517
    PID: 555542726 has 5 field diffs  and GEOM diff
ERROR processing mpa 7517:  'NoneType' object has no attribute 'hexewkb'
2/17 adding/updating wdpa_pid 555542727 with mpa_id 7518
    PID: 555542727 has 4 field diffs 
ERROR processing mpa 7518:  'NoneType' object has no attribute 'hexewkb'
3/17 adding/updating wdpa_pid 2100 with mpa_id 5861
    PID: 2100 has 1 field diffs 
ERROR processing mpa 5861:  'NoneType' object has no attribute 'hexewkb'
4/17 adding/updating wdpa_pid 3031 with mpa_id 5878
    PID: 3031 has 1 field diffs 
ERROR processing mpa 5878:  'NoneType' object has no attribute 'hexewkb'
5/17 adding/updating wdpa_pid 317276 with mpa_id 7169
    PID: 317276 has 1 field diffs 
ERROR processing mpa 7169:  'NoneType' object has no attribute 'hexewkb'
6/17 adding/updating wdpa_pid 8643 with mpa_id 60005993
    PID: 8643 has 3 field diffs  and GEOM diff
ERROR processing mpa 60005993:  'NoneType' object has no attribute 'h

True

In [130]:
updatepolys = Wdpa2018Poly.objects.filter(wdpa_pid__in=allpid2update)
merge.updateMpasFromWdpaQueryset(qs=updatepolys, poly=True, logfile=updatepoly_log, dryrun=True)

1/187 adding/updating wdpa_pid 555511959 with mpa_id 5388
    PID: 555511959 has 10 field diffs  and GEOM diff
Carto Error for mpa_id 5388: Access denied
2/187 adding/updating wdpa_pid 555511959 with mpa_id 7705388
    PID: 555511959 has 12 field diffs  and GEOM diff
Carto Error for mpa_id 7705388: Access denied
3/187 adding/updating wdpa_pid 30000 with mpa_id 1758
    PID: 30000 has 8 field diffs  and GEOM diff
Carto Error for mpa_id 1758: Access denied
4/187 adding/updating wdpa_pid 555511989 with mpa_id 5416
    PID: 555511989 has 10 field diffs  and GEOM diff
Carto Error for mpa_id 5416: Access denied
5/187 adding/updating wdpa_pid 555511968 with mpa_id 5395
    PID: 555511968 has 11 field diffs  and GEOM diff
Carto Error for mpa_id 5395: Access denied
6/187 adding/updating wdpa_pid 30012 with mpa_id 1759
    PID: 30012 has 11 field diffs  and GEOM diff
Carto Error for mpa_id 1759: Access denied
7/187 adding/updating wdpa_pid 30012 with mpa_id 67701759
    PID: 30012 has 11 field d

    PID: 1968 has 9 field diffs  and GEOM diff
Carto Error for mpa_id 253: Access denied
58/187 adding/updating wdpa_pid 8871 with mpa_id 695
    PID: 8871 has 9 field diffs  and GEOM diff
Carto Error for mpa_id 695: Access denied
59/187 adding/updating wdpa_pid 8102 with mpa_id 67700615
    PID: 8102 has 13 field diffs  and GEOM diff
Carto Error for mpa_id 67700615: Access denied
60/187 adding/updating wdpa_pid 2349 with mpa_id 278
    PID: 2349 has 7 field diffs  and GEOM diff
Carto Error for mpa_id 278: Access denied
61/187 adding/updating wdpa_pid 198427 with mpa_id 3532
    PID: 198427 has 10 field diffs  and GEOM diff
Carto Error for mpa_id 3532: Access denied
62/187 adding/updating wdpa_pid 198427 with mpa_id 67703532
    PID: 198427 has 11 field diffs  and GEOM diff
Carto Error for mpa_id 67703532: Access denied
63/187 adding/updating wdpa_pid 198425 with mpa_id 3531
    PID: 198425 has 12 field diffs  and GEOM diff
Carto Error for mpa_id 3531: Access denied
64/187 adding/updat

Carto Error for mpa_id 6058: Access denied
113/187 adding/updating wdpa_pid 101486 with mpa_id 2622
    PID: 101486 has 8 field diffs  and GEOM diff
Carto Error for mpa_id 2622: Access denied
114/187 adding/updating wdpa_pid 20432 with mpa_id 1499
    PID: 20432 has 12 field diffs  and GEOM diff
Carto Error for mpa_id 1499: Access denied
115/187 adding/updating wdpa_pid 8906 with mpa_id 709
    PID: 8906 has 3 field diffs  and GEOM diff
Carto Error for mpa_id 709: Access denied
116/187 adding/updating wdpa_pid 8906 with mpa_id 67700709
    PID: 8906 has 8 field diffs  and GEOM diff
Carto Error for mpa_id 67700709: Access denied
117/187 adding/updating wdpa_pid 12871 with mpa_id 1008
    PID: 12871 has 3 field diffs  and GEOM diff
Carto Error for mpa_id 1008: Access denied
118/187 adding/updating wdpa_pid 303689 with mpa_id 3804
    PID: 303689 has 6 field diffs  and GEOM diff
Carto Error for mpa_id 3804: Access denied
119/187 adding/updating wdpa_pid 555511994 with mpa_id 5420
    PID:

Carto Error for mpa_id 4854: Access denied
169/187 adding/updating wdpa_pid 555511967 with mpa_id 5394
    PID: 555511967 has 11 field diffs  and GEOM diff
Carto Error for mpa_id 5394: Access denied
170/187 adding/updating wdpa_pid 555511967 with mpa_id 67705394
    PID: 555511967 has 13 field diffs  and GEOM diff
Carto Error for mpa_id 67705394: Access denied
171/187 adding/updating wdpa_pid 555511976 with mpa_id 5403
    PID: 555511976 has 10 field diffs  and GEOM diff
Carto Error for mpa_id 5403: Access denied
172/187 adding/updating wdpa_pid 8609 with mpa_id 635
    PID: 8609 has 9 field diffs  and GEOM diff
Carto Error for mpa_id 635: Access denied
173/187 adding/updating wdpa_pid 8609 with mpa_id 67700635
    PID: 8609 has 11 field diffs  and GEOM diff
Carto Error for mpa_id 67700635: Access denied
174/187 adding/updating wdpa_pid 8784 with mpa_id 681
    PID: 8784 has 11 field diffs  and GEOM diff
Carto Error for mpa_id 681: Access denied
175/187 adding/updating wdpa_pid 198424 

True

In [5]:
Mpa.objects.filter(wdpa_pid__isnull=False).update(
    no_take_wdpa=F('no_take'), no_take_area_wdpa=F('no_take_area')
)

13554

In [9]:
nt = ('All', 'Part', 'None')
ntupdate = Mpa.objects.filter(wdpa_pid__isnull=False).filter(
    (~Q(no_take_wdpa__in=('All')) & Q(no_take_mpatlas__in=('All')))
    | (~Q(no_take_wdpa__in=('All', 'Part')) & Q(no_take_mpatlas__in=('All', 'Part')))
    | (Q(no_take_wdpa__in=('Not Applicable')) & Q(no_take_mpatlas__in=('All', 'Part', 'None')))
)
ntupdate.update(no_take=F('no_take_mpatlas'))

16

In [16]:
ntareaupdate = Mpa.objects.filter(wdpa_pid__isnull=False).filter(
    Q(no_take_area_mpatlas__isnull=False) &
    (
        Q(no_take_area_wdpa__isnull=True)
        | Q(no_take_area_mpatlas__gt=F('no_take_area_wdpa'))
    )
)
ntareaupdate.update(no_take_area=F('no_take_area_mpatlas'))

18

In [12]:
for n in ntupdate:
    print(n.mpa_id, n.wdpa_id, n.wdpa_pid, n.name, n.designation, n.designation_eng, n.no_take_wdpa, n.no_take_mpatlas)

6989 303130 303130 Tung Ping Chau Marine Park Marine Park Not Reported Part
67704213 306779 306779 Cabo Pulmo Parque Nacional National Park Not Reported All
4210 306775 306775_A Arrecife Alacranes Parque Nacional National Park Not Reported Part
7705345 902309 902309 Parque Nacional Bahía de Loreto Ramsar Site, Wetland of International Importance Ramsar Site, Wetland of International Importance Not Reported Part
5388 555511959 555511959 Southeast Misool (Raja Ampat) Kawasan Konservasi Perairan Daerah Locally Managed Marine Area Not Reported Part
2724 107706 107706_A Islas Marietas Parque Nacional National Park Not Reported Part
4216 306808 306808_A Isla San Pedro Mártir Reserva de la Biósfera Biosphere Reserve Not Reported Part
4221 306850 306850_A Sistema Arrecifal Veracruzano Parque Nacional National Park Not Reported Part
6823 115671 115671 Cape D'Aguilar Marine Reserve Marine Reserve Not Reported All
4872 342345 342345 Isla Guadalupe Reserva de la Biósfera Biosphere Reserve Not Repo

In [17]:
for n in ntareaupdate:
    print(n.mpa_id, n.wdpa_id, n.wdpa_pid, n.name, n.designation, n.designation_eng, n.no_take_wdpa, n.no_take_mpatlas, n.no_take_area_wdpa, n.no_take_area_mpatlas)

67704213 306779 306779 Cabo Pulmo Parque Nacional National Park Not Reported All 0.0 71.11
4210 306775 306775_A Arrecife Alacranes Parque Nacional National Park Not Reported Part 0.0 316.69
5388 555511959 555511959 Southeast Misool (Raja Ampat) Kawasan Konservasi Perairan Daerah Locally Managed Marine Area Not Reported Part 0.0 1212.1
2724 107706 107706_A Islas Marietas Parque Nacional National Park Not Reported Part 0.0 0.78
4216 306808 306808_A Isla San Pedro Mártir Reserva de la Biósfera Biosphere Reserve Not Reported Part 0.0 11.11
4221 306850 306850_A Sistema Arrecifal Veracruzano Parque Nacional National Park Not Reported Part 0.0 11.12
4872 342345 342345 Isla Guadalupe Reserva de la Biósfera Biosphere Reserve Not Reported Part 0.0 0.4
1918 61409 61409 El Vizcaíno UNESCO-MAB Biosphere Reserve UNESCO-MAB Biosphere Reserve Not Reported Part 0.0 46.77
7705388 555511959 555511959 Southeast Misool (Raja Ampat) Kawasan Konservasi Perairan Daerah Locally Managed Marine Area Not Reported

In [19]:
Mpa.objects.filter(mpa_id__in=(6058, 5995, 1499, 918, 917, 710)).update(implemented=True)

6